# Water Safety App for Dogs 🐶

This notebook evaluates water safety for swimming based on chlorophyll and turbidity levels. It checks whether the water is safe for dogs to bathe in, using real-time data from an API (AquaSat and geocoding services).

In [1]:
import requests
import folium
from flask import Flask, render_template
import plotly.graph_objects as go

In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from ttkbootstrap import Style
from PIL import Image, ImageTk
import ee
from datetime import datetime, timedelta 
from opencage.geocoder import OpenCageGeocode

In [ ]:
# App Using Tkinter 

# Function to get coordinates from OpenCage API
def get_coordinates(address):
    api_key = os.getenv('OpenCage_APIKEY')
    url = f"https://api.opencagedata.com/geocode/v1/json?q={address}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        if result['results']:
            lat = result['results'][0]['geometry']['lat']
            lng = result['results'][0]['geometry']['lng']
            return lat, lng
        else:
            messagebox.showerror("Error", "Address not found.")
            return None
    else:
        messagebox.showerror("Error", f"Error: {response.status_code}")
        return None

# Function to fetch water quality data from Earth Engine and AquaSat
def fetch_water_quality(lat, lon):
    # Initialize Earth Engine
    ee.Initialize()

    # Define a point of interest
    point = ee.Geometry.Point([lon, lat])
    
    # Define time range for the last 7 days
    end_date = datetime.utcnow()
    start_date = end_date - timedelta(days=7)
    
    # Fetch Sentinel-3 OLCI data for the point within the last 7 days
    sentinel3 = ee.ImageCollection("COPERNICUS/S3/OLCI") \
        .filterBounds(point) \
        .filterDate(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')) \
        .select(['Oa02_radiance', 'Oa03_radiance', 'Oa04_radiance', 'Oa05_radiance', 'Oa08_radiance'])
    
    # Check if any images exist
    if sentinel3.size().getInfo() == 0:
        return None, "No Sentinel-3 data available for the given location in the last 7 days."
    
    # Compute mean radiance values for each band over the time range
    thresholds = {'Oa02_radiance': 0.05, 'Oa03_radiance': 0.05, 'Oa04_radiance': 0.05, 'Oa05_radiance': 0.05, 'Oa08_radiance': 0.05}
    flagged_dates = []

    # Loop through images
    images = sentinel3.toList(sentinel3.size())
    for i in range(images.size().getInfo()):
        image = ee.Image(images.get(i))
        mean_values = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=300
        ).getInfo()

        # Check each band against the thresholds
        for band, threshold in thresholds.items():
            if band in mean_values and mean_values[band] is not None and mean_values[band] > threshold:
                date = image.date().format('YYYY-MM-dd').getInfo()
                flagged_dates.append((date, band))
                break  # If one band is flagged, no need to check others

    # Return flagged dates and bands if any
    if flagged_dates:
        return flagged_dates, None
    else:
        return None, "No flags detected in the last 7 days."

# Function to fetch nearby beaches using Google Places API
def fetch_nearby_beaches(lat, lon, api_key):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lon}&radius=5000&keyword=beach&key={api_key}"
    response = requests.get(url)
    places = response.json().get("results", [])

    nearby_beaches = []
    for place in places[:5]:  # Limit to 5 results
        name = place.get('name')
        vicinity = place.get('vicinity', 'Unknown location')
        nearby_beaches.append(f"{name} - {vicinity}")
    
    return nearby_beaches

# Function to handle button click
def on_search_button_click():
    address = entry.get()  # Get the address from the input field
    if not address:
        messagebox.showerror("Input Error", "Please enter an address.")
        return
    
    # Get coordinates from address
    coordinates = get_coordinates(address)
    if coordinates:
        lat, lng = coordinates
        result_label.config(text=f"Coordinates: Latitude: {lat}, Longitude: {lng}")
        
        # Fetch water quality data
        try:
            flagged_data, message = fetch_water_quality(lat, lng)
            if flagged_data:
                flagged_list = "\n".join([f"{date} - {band}" for date, band in flagged_data])
                messagebox.showwarning("Water Quality Alert", f"Wait! 🐶 \n\n Water near your location may have increased vegetation and murky conditions. \n\n Check your local health department's website for updates on water quality near you.")
                
                # Fetch nearby beaches
                google_api_key = "google_api_key"  
                nearby_beaches = fetch_nearby_beaches(lat, lng, google_api_key)
                if nearby_beaches:
                    messagebox.showinfo("Nearby Beaches", " \n".join(nearby_beaches))
                else:
                    messagebox.showinfo("Nearby Beaches", "No nearby beaches found.")
            else:
                messagebox.showinfo("Water Quality", "Water quality is safe in the last 7 days.")
            root.destroy()  # Stop execution by closing the app
        except Exception as e:
            messagebox.showerror("Error", f"Error fetching water quality: {e}")

# Set up the Tkinter window with ttkbootstrap
root = tk.Tk()
style = Style('superhero')  # You can change the theme here
style.master = root

root.title("DjurSafe")
root.geometry("400x600")

# Splash screen elements
title_label = ttk.Label(root, text="DjurSafe", font=("Helvetica", 45, "bold"), anchor="center", background=style.lookup("TFrame", "background"))
title_label.pack(pady=10)

description_label = ttk.Label(
    root, 
    text="Out and about? \n\n Enter an address near you for water quality updates in the last 7 days:", 
    wraplength=350, 
    font=("Helvetica", 25),
    anchor="center",
    justify="center",
    background=style.lookup("TFrame", "background")
)
description_label.pack(pady=10)

# Add dog icon
try:
    dog_image = Image.open("/Users/jt/Downloads/dog.png")
    dog_image = dog_image.resize((200, 200))  # Resize if necessary
    dog_photo = ImageTk.PhotoImage(dog_image)
    dog_label = ttk.Label(root, image=dog_photo)
    dog_label.pack(pady=10)
except Exception as e:
    messagebox.showerror("Error", f"Failed to load image: {e}")

# Input field for the address
entry = ttk.Entry(root, width=50)
entry.pack(pady=10)

# Search button
search_button = ttk.Button(root, text="Search", command=on_search_button_click)
search_button.pack(pady=10)

# Result label to show coordinates and water quality status
result_label = ttk.Label(root, text="Coordinates and water quality will be displayed here.", wraplength=350)
result_label.pack(pady=10)

# Main loop
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.12/tkinter/__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/rk/7cxcvzcs2pg_yf9qdjfd6byh0000gn/T/ipykernel_1546/3650246392.py", line 92, in on_search_button_click
    coordinates = get_coordinates(address)
                  ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/rk/7cxcvzcs2pg_yf9qdjfd6byh0000gn/T/ipykernel_1546/3650246392.py", line 7, in get_coordinates
    response = requests.get(url)
               ^^^^^^^^
NameError: name 'requests' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.12/tkinter/__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/rk/7cxcvzcs2pg_yf9qdjfd6byh0000gn/T/ipykernel_1546/3650246392.py", line 92, in on_search_button_click
    coordinates = get_coordinates(address)
 